# **Declustering**
***

### **Editado por: Kevin Alexander Gómez**
#### Contacto: kevinalexandr19@gmail.com | [Linkedin](https://www.linkedin.com/in/kevin-alexander-g%C3%B3mez-2b0263111/) | [Github](https://github.com/kevinalexandr19)
***

### **Descripción**
En este tutorial, aprenderás el concepto de **declustering** en Geoestadística, a través del uso de la librería `geostatspy`.

Es necesario haber llevado previamente los tutoriales de [Pandas](1c_pandas.ipynb) y [Matplotlib](1d_matplotlib.ipynb).

Este Notebook es parte del proyecto [**Python para Geólogos**](https://github.com/kevinalexandr19/manual-python-geologia), y ha sido creado con la finalidad de facilitar el aprendizaje en Python para estudiantes y profesionales en el campo de la Geología.

***

## **1. Representatividad en el muestreo geoestadístico**

Se dice que una muestra es **representativa** cuando refleja las características esenciales de la población de la cuál fue extraída.

En general, debemos asumir que todas las muestras tomadas del campo se encuentran sesgadas de alguna forma.

Si tuviéramos que realizar un muestreo tomando en cuenta la representatividad de las muestras, teóricamente, tendríamos dos opciones:
- Realizar un **muestreo aleatorio**, en donde asumimos que cada elemento de la población tiene la misma probabilidad de ser extraída.
- Realizar un **muestreo sistemático**, en donde las muestras son extraídas a intervalos regulares (igualmente espaciadas).

## **2. ¿Qué es declustering?**

En la actividad de muestreo, es frecuente encontrar áreas con una mayor concentración de muestras.\
Esta práctica conlleva a un sesgo en la estadística general de los datos debido a que la distribución **irregular** de las muestras reduce la representatividad del volumen de interés.

Para tratar este sesgo en la toma de muestras, podemos hacer uso del **declustering** o **desagrupamiento**.\
Se le asigna a cada dato un **peso** o **ponderación** basada en su cercanía a las muestras circundantes. Las ponderaciones son mayores a 0 y en total suman 1.

Para evaluar la cercanía, se utiliza una malla que divide el área en celdas con un tamaño específico.\
Cada celda puede contener varias o ninguna de las muestras, mientras más muestras tenga una celda, menor será la ponderación asignada.\
De la misma forma, una muestra alejada de las demás tendrá una ponderación más alta que aquellas que se encuentren agrupadas.

<img src="resources/declustering_weights.png" width="600"/>

Si el tamaño de la celda fuera equivalente al tamaño de la malla, el promedio de los datos sería equivalente al promedio sin desagrupar.\
Si por otro lado, el tamaño de la celda fuera extremadamente pequeño, el promedio de los datos también sería equivalente al promedio sin desagrupar.\
Por lo tanto, existe un tamaño de celda óptimo entre estos extremos que se usará para desagrupar los datos.

También debemos tener en cuenta que la ubicación de la malla también influye en la ponderación individual de cada muestra.\
Para resolver este problema, se pueden tomar varias ubicaciones aleatorias y se promedian las ponderaciones individuales asignadas a cada muestra.

Una vez se asignan las ponderaciones de desagrupamiento a cada muestra, podemos obtener medidas estadísticas desagrupadas como el **promedio**, **varianza**, **covarianza**, etc.

***

## **3. Declustering con Python**

Empezaremos importando las librerías que utilizaremos en este tutorial:

In [ ]:
# Librería geoestadística
import geostatspy.GSLIB as GSLIB          # GSLIB: herramientas, visualizador y wrapper
import geostatspy.geostats as geostats    # Métodos de GSLIB convertidos a Python

# Librerías fundamentales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Funciones estadísticas
from statsmodels.stats.weightstats import DescrStatsW

Y abriremos el archivo `data_sesgada.csv`, que contiene la información a desagrupar.

In [ ]:
data = pd.read_csv("files/data_sesgada.csv")

In [ ]:
data.head()

### **3.1. Descripción de los datos**
- `X`, `Y`: coordenadas
- `Facies`: 1 para arenisca y 0 para intercalaciones de arenisca y lutita
- `Porosidad`: porosidad en fracción (%)
- `Permeabilidad` : permeabilidad en miliDarcy (mDarcy)

In [ ]:
# Resumen estadístico
data.describe()

### **3.2. Especificando el área de interés**

Es común delimitar manualmente el rango de las coordenadas X e Y. También estableceremos un rango para la columna de `Porosidad` y un mapa de colores para la visualización.

In [ ]:
# Coordenadas
xmin, xmax = 0., 1000.
ymin, ymax = 0., 1000.

# Porosidad
pormin, pormax = 0.05, 0.25

# Mapa de colores
cmap = plt.cm.inferno

Para mostrar el área de interés en un gráfico, crearemos una figura similar al `locmap` de GSLIB:

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(8, 6), subplot_kw={"aspect": 1})

# Diagrama de dispersión
im = ax.scatter(data=data, x="X", y="Y", c="Porosidad", s=40, cmap=cmap, edgecolor="black", alpha=0.8)
im.set_clim(pormin, pormax)

# Barra de colores
cbar = fig.colorbar(im, ax=ax)
cbar.set_label("Porosidad (%)", fontsize=16, rotation=270, labelpad=25)

# Límites de la figura
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

# Título y nombres
ax.set_title("Data - Porosidad", fontsize=18)
ax.set_xlabel("X (m)", fontsize=14)
ax.set_ylabel("Y (m)", fontsize=14)

plt.show()

Podemos notar que en las regiones de alta porosidad existe un mayor número de muestras.\
Esto se puede considerar como un **muestreo preferencial** o **selectivo**.

Debido a este sesgo, no podemos usar la estadística general para representar esta región.\
Debemos de realizar una **correción por agrupamiento** de las muestras en las regiones de alta porosidad.

En este caso, utilizaremos el **desagrupamiento por celdas**, y trataremos de minimizar la **media desagrupada**.\
Visualmente, podemos notar que un tamaño de celda adecuado debería estar entre 100 y 200 metros.

### **3.3. Desagrupando los datos**

Para realizar el desagrupamiento, usaremos la función `declus` reimplementada de GSLIB en Python, a través del módulo `geostats`.

In [ ]:
# Detalles de la función
geostats.declus

Como podemos ver, la función tiene los siguientes parámetros:
- `df`: el DataFrame con la información
- `xcol`, `ycol`: las columnas de coordenadas x e y
- `vcol`: la columna que contiene la variable de interés
- `iminmax`: puede ser `0`/`False` si se usa un tamaño de celda que maximice la media desagrupada o `1`/`True` si se usa un tamaño que minimice la media desagrupada.
- `noff`: número de ubicaciones aleatorias para la malla
- `ncell`: número de tamaños de celda a probar por cada malla
- `cmin`: tamaño mínimo de celda
- `cmax`: tamaño máximo de celda

Probaremos con un amplio rango de tamaño de celdas, de 10 m a 2000 m, y eligiremos aquel tamaño que minimice la media desagrupada.\
También usaremos 10 ubicaciones aleatorias de malla y 100 tamaños de celda a probar por cada malla.

El resultado de la función `declus` está compuesto por:
- `wts`: un arreglo que contiene las ponderaciones desagrupadas de cada dato (la suma es equivalente al número de datos, el valor de 1 indica un peso nominal)
- `cell_sizes`: un arreglo con los tamaños de celda considerados
- `dmeans`: un arreglo con las medias desagrupadas, calculadas por cada tamaño de celda en `cell_sizes`

Ahora, usaremos la función para obtener las ponderaciones y generar un gráfico para elegir el tamaño de celda óptimo.

In [ ]:
wts, cell_sizes, dmeans = geostats.declus(data, "X", "Y", "Porosidad", 1, 10, 100, 10, 2000)

Creamos una nueva columna en la data con las ponderaciones:

In [ ]:
data["wts"] = wts
data.head()

Y ahora graficaremos la distribución de las ponderaciones sobre el área de interés:
> Estableceremos un rango de 0.25 a 4 para los valores de ponderación.

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(8, 6), subplot_kw={"aspect": 1})

# Diagrama de dispersión
im = ax.scatter(data=data, x="X", y="Y", c="wts", s=40, cmap=cmap, edgecolor="black", alpha=0.8)
im.set_clim(0.25, 4)

# Barra de colores
cbar = fig.colorbar(im, ax=ax)
cbar.set_label("Ponderaciones", fontsize=16, rotation=270, labelpad=25)

# Límites de la figura
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

# Título y nombres
ax.set_title("Data - Ponderaciones", fontsize=18)
ax.set_xlabel("X (m)", fontsize=14)
ax.set_ylabel("Y (m)", fontsize=14)

plt.show()

Observamos que las ponderaciones varían de acuerdo a la densidad de muestras en la región, por lo tanto, hemos conseguido desagrupar las muestras.

### **3.4. Cuadro resumen del desagrupamiento**

Ahora, crearemos una figura resumen en la cual graficaremos lo siguiente:
- El área de interés con las ponderaciones asignadas,
- Un histograma mostrando la distribución de ponderaciones,
- Una comparación entre las distribuciones de porosidad para las muestras sin desagrupar y desagrupadas.

In [ ]:
# Figura principal
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 15))

# 1. Área de interés con ponderaciones
# Diagrama de dispersión
im1 = ax1.scatter(data=data, x="X", y="Y", c="wts", s=40, cmap=cmap, edgecolor="black", alpha=0.8)
im1.set_clim(0.25, 4)

# Barra de colores
cbar = fig.colorbar(im1, ax=ax1)
cbar.set_label("Ponderaciones", fontsize=16, rotation=270, labelpad=25)

# Límites de la figura
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set(aspect=1)

# Título y nombres
ax1.set_title("Data - Ponderaciones", fontsize=18)
ax1.set_xlabel("X (m)", fontsize=14)
ax1.set_ylabel("Y (m)", fontsize=14)


# 2. Histograma de ponderaciones
# Histograma
ax2.hist(data=data, x="wts", bins=20, color="darkorange", edgecolor="black")
ax2.margins(x=0)

# Título y nombres
ax2.set_title("Ponderaciones de desagrupamiento", fontsize=18)
ax2.set_xlabel("Ponderaciones", fontsize=14)
ax2.set_ylabel("Frecuencia", fontsize=14)


# 3. Comparación entre distribuciones de porosidad
# Figura izquierda - Porosidad sin desagrupar
# Histograma
ax3.hist(data=data, x="Porosidad", bins=20, color="darkorange", edgecolor="black")
ax3.set_xlim(pormin, pormax)

# Título y nombres
ax3.set_title("Porosidad sin desagrupar", fontsize=18)
ax3.set_xlabel("Porosidad (%)", fontsize=14)
ax3.set_ylabel("Frecuencia", fontsize=14)

# Figura derecha - Porosidad desagrupada
# Histograma
ax4.hist(data=data, x="Porosidad", weights="wts", bins=20, color="darkorange", edgecolor="black")
ax4.set_xlim(pormin, pormax)

# Título y nombres
ax4.set_title("Porosidad desagrupada", fontsize=18)
ax4.set_xlabel("Porosidad (%)", fontsize=14)
ax4.set_ylabel("Frecuencia", fontsize=14)

# Ajuste de altura de ambos histogramas
ax4.get_shared_y_axes().join(ax3, ax4)

plt.show()

También generaremos un resumen de la variación en la media de porosidad al desagrupar los datos:

In [ ]:
mean = np.average(data["Porosidad"].values)
dmean = np.average(data["Porosidad"].values, weights=data["wts"].values)
correction = (mean - dmean) / mean

print(f"La media de porosidad sin desagrupar es de {mean:.3f}")
print(f"La media de porosidad desagrupada es de {dmean:.3f}")
print(f"Corrección de {correction:.2%}")

### **3.5. Tamaño de celda óptimo**

Ahora, crearemos un gráfico mostrando la **media desagrupada de porosidad** vs. el **tamaño de celda de desagrupamiento** a través de las 100 repeticiones que se realizaron.\
Recordemos que cuando el tamaño de celda es demasiado grande o demasiado pequeño, la media desagrupada es equivalente a la media sin desagrupar.

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(10, 7))

# Diagrama de dispersión
ax.scatter(cell_sizes, dmeans, s=30, alpha=0.8, edgecolor="black", facecolor="darkorange")

# Ticks del eje x
ax.set_xticks(np.linspace(0, 2000, 11))

# Límites de la figura
ax.margins(x=0)
ax.set_ylim(0.10, 0.16)

# Título y nombres
ax.set_title("Media desagrupada de Porosidad vs. Tamaño de celda", fontsize=18)
ax.set_xlabel("Tamaño de celda (m)", fontsize=14)
ax.set_ylabel("Media desagrupada de Porosidad (%)", fontsize=14)

plt.show()

Notamos que el tamaño de celda óptimo se encuentra aproxidamente en 200 metros. Graficaremos unas líneas adicionales en la figura:

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(10, 7))

# Diagrama de dispersión
ax.scatter(cell_sizes, dmeans, s=30, alpha=0.8, edgecolor="black", facecolor="darkorange")

# Ticks del eje x
ax.set_xticks(np.linspace(0, 2000, 11))

# Límites de la figura
ax.margins(x=0)
ax.set_ylim(0.10, 0.16)

# Título y nombres
ax.set_title("Media desagrupada de Porosidad vs. Tamaño de celda", fontsize=18)
ax.set_xlabel("Tamaño de celda (m)", fontsize=14)
ax.set_ylabel("Media desagrupada de Porosidad (%)", fontsize=14)

# Tamaño de celda óptimo
ax.plot([0, 2000], [mean, mean], c="black")
ax.plot([200, 200], [0.10, 0.16], c="black", ls="dashed")

# Texto en la figura
ax.text(300, 0.136, "Media sin desagrupar de Porosidad", fontsize=12)
ax.text(500, 0.118, "Media desagrupada de Porosidad", fontsize=12)
ax.text(230, 0.151, "Tamaño de\ncelda óptimo", fontsize=12)

plt.show()

### **3.6. Estadística descriptiva de los datos desagrupados**

Si bien podemos calcular la media, varianza y desviación estándar manualmente, también podemos utilizar la función `DescrStatsW` del módulo `statsmodels.stats.weights`.\
Esta función nos permite agregar ponderaciones a un conjunto de datos a través de los siguientes parámetros:
- `data`: es el arreglo que contiene los datos
- `weights`: son las ponderaciones a utilizar para cada dato

Asignaremos el conjunto ponderado a una variable llamada `ddata`:

In [ ]:
ddata = DescrStatsW(data=data["Porosidad"].values, weights=data["wts"])

Y ahora, para finalizar, generaremos un resumen estadístico de los datos sin desagrupar y desagrupados de Porosidad:

In [ ]:
# Cálculo manual de los datos sin desagrupar
mean = np.average(data["Porosidad"].values)
var = np.var(data["Porosidad"].values)
std = np.std(data["Porosidad"].values)

# Resumen estadístico
print(f"Estadística sin desagrupar - Porosidad")
print(f"    Media: {mean:.3f}")
print(f"    Varianza: {var:.5f}")
print(f"    Desviación estándar: {std:.3f}\n")

print(f"Estadística desagrupada - Porosidad")
print(f"    Media: {ddata.mean:.3f}")
print(f"    Varianza: {ddata.var:.5f}")
print(f"    Desviación estándar: {ddata.std:.3f}")

En conclusión, realizar un **declustering** o **desagrupamiento** de los datos nos permite corregir el sesgo de muestreo.